In [10]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install transformers==4.41.0
!pip install biopython
!pip install tqdm
!pip install pandas
!pip install numpy
!pip install scikit-learn


Looking in indexes: https://download.pytorch.org/whl/cu121
  Using cached transformers-4.41.0-py3-none-any.whl.metadata (43 kB)
  Using cached huggingface_hub-0.36.0-py3-none-any.whl.metadata (14 kB)
  Using cached regex-2025.11.3-cp310-cp310-win_amd64.whl.metadata (41 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached transformers-4.41.0-py3-none-any.whl (9.1 MB)
Using cached huggingface_hub-0.36.0-py3-none-any.whl (566 kB)
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 25.1 MB/s  0:00:00
Using cached regex-2025.11.3-cp310-cp310-win_amd64.whl (277 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)

   ---------- -----------------------------  3/12 [regex]
   ----------------------- ----------------  7/12 [certifi]
   ------------------------------ ---------  9/12 [huggingface-hub]
   --------------------------------- ------ 10/12 [tokenizers]
   ----------------------------------

In [11]:
import torch
print("CUDA:", torch.cuda.is_available())
print("Device:", torch.cuda.get_device_name(0))

CUDA: True
Device: NVIDIA GeForce RTX 4060


In [13]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import random
import numpy as np
import pandas as pd
from typing import List

import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForMaskedLM


# =====================================================
# 0) 경로 설정 (🔥 지예님 PC 기준)
# =====================================================
BASE = r"C:\Users\USER\Desktop\전지예"

GRCH38_CSV = os.path.join(BASE, "grch38_windows_seq.csv")
CHM13_CSV  = os.path.join(BASE, "chm13_windows_seq.csv")

TEST_PATH        = os.path.join(BASE, "test.csv")
SAMPLE_SUB_PATH  = os.path.join(BASE, "sample_submission.csv")
OUT_PATH         = os.path.join(BASE, "submission_infonce_v4.csv")


# =====================================================
# 1) InfoNCE 학습 설정
# =====================================================
SEED = 42
MODEL_ID = "InstaDeepAI/nucleotide-transformer-v2-500m-multi-species"

MAX_EXT_SEQ = 120000     # 외부 데이터 샘플링 최대 개수
NUM_PAIRS   = 30000      # InfoNCE pair 개수

MAX_LENGTH = 512
LAST_N_LAYERS = 4
OUTPUT_DIM = 512

TRAIN_EPOCHS = 3
BATCH_SIZE_TR = 32
BATCH_SIZE_INFER = 32

LR_HEAD = 1e-4
WEIGHT_DECAY = 1e-4
TEMPERATURE = 0.07
USE_FP16 = True


# =====================================================
# 2) Utility
# =====================================================
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


def l2_normalize(x, eps=1e-12):
    return x / x.norm(p=2, dim=-1, keepdim=True).clamp(min=eps)


_rc_map = str.maketrans("ACGT", "TGCA")
def reverse_complement(seq):
    return seq.translate(_rc_map)[::-1]


def apply_snv(seq, k=1):
    bases = ["A", "C", "G", "T"]
    s = list(seq)
    idxs = random.sample(range(len(s)), k)
    for idx in idxs:
        orig = s[idx]
        s[idx] = random.choice([b for b in bases if b != orig])
    return "".join(s)


def generate_pairs(seqs: List[str], num_pairs: int):
    pairs = []
    for _ in range(num_pairs):
        anchor = random.choice(seqs)

        # 길면 MAX_LENGTH로 랜덤 슬라이스
        if len(anchor) > MAX_LENGTH:
            st = random.randint(0, len(anchor) - MAX_LENGTH)
            anchor = anchor[st:st + MAX_LENGTH]

        r = random.random()
        if r < 0.4:
            pos = anchor
        elif r < 0.8:
            pos = apply_snv(anchor, random.randint(1, 2))
        else:
            pos = reverse_complement(apply_snv(anchor))

        pairs.append([anchor, pos])

    return pairs


# =====================================================
# 3) RobustModel
# =====================================================
class RobustModel(nn.Module):
    def __init__(self, hidden, last_n, out_dim):
        super().__init__()
        self.last_n = last_n
        self.layer_weights = nn.Parameter(torch.zeros(last_n))

        self.proj = nn.Sequential(
            nn.Linear(hidden, hidden * 2),
            nn.LayerNorm(hidden * 2),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(hidden * 2, out_dim)
        )

    def forward(self, hidden_states, mask):
        stack = torch.stack(hidden_states[-self.last_n:], 0)
        w = F.softmax(self.layer_weights, dim=0).view(-1, 1, 1, 1)

        feat = (stack * w).sum(0)
        mask = mask.unsqueeze(-1).float()

        summed = (feat * mask).sum(1)
        denom = mask.sum(1).clamp(min=1e-9)

        mean_emb = summed / denom
        return l2_normalize(self.proj(mean_emb))


# =====================================================
# 4) Main
# =====================================================
def main():

    print("🔥 GPU:", torch.cuda.get_device_name(0))
    set_seed(SEED)
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # -------------------------------------------------
    # 1) 이미 전처리된 CSV 불러오기 (GRCh38 + CHM13)
    # -------------------------------------------------
    print("\n📌 외부데이터 로드 (GRCh38 + CHM13)")
    df1 = pd.read_csv(GRCH38_CSV)
    df2 = pd.read_csv(CHM13_CSV)

    combined = pd.concat([df1, df2], ignore_index=True)
    print("👉 전체 외부 데이터 개수:", len(combined))

    ext_sequences = combined["seq"].astype(str).tolist()

    # 샘플링
    if len(ext_sequences) > MAX_EXT_SEQ:
        idx = np.random.choice(len(ext_sequences), MAX_EXT_SEQ, replace=False)
        ext_sequences = [ext_sequences[i] for i in idx]
        print("🔻 샘플링:", len(ext_sequences))

    # InfoNCE 데이터 생성
    print("\n📌 InfoNCE pair 생성")
    pair_data = generate_pairs(ext_sequences, NUM_PAIRS)

    # -------------------------------------------------
    # 2) test.csv 로드
    # -------------------------------------------------
    test_df = pd.read_csv(TEST_PATH)
    test_sequences = test_df["seq"].astype(str).tolist()
    print("👉 test 시퀀스 개수:", len(test_sequences))

    # -------------------------------------------------
    # 3) 백본 모델 로드
    # -------------------------------------------------
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
    backbone = AutoModelForMaskedLM.from_pretrained(MODEL_ID, trust_remote_code=True)
    backbone = backbone.to(device).eval()

    for p in backbone.parameters():
        p.requires_grad = False

    # -------------------------------------------------
    # 4) 헤드 모델
    # -------------------------------------------------
    model = RobustModel(backbone.config.hidden_size, LAST_N_LAYERS, OUTPUT_DIM).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LR_HEAD, weight_decay=WEIGHT_DECAY)
    scaler = torch.cuda.amp.GradScaler(enabled=USE_FP16)

    # -------------------------------------------------
    # 5) InfoNCE 학습
    # -------------------------------------------------
    print("\n🚀 InfoNCE 학습 시작")

    for epoch in range(TRAIN_EPOCHS):
        random.shuffle(pair_data)
        epoch_losses = []

        for i in tqdm(range(0, len(pair_data), BATCH_SIZE_TR), desc=f"EPOCH {epoch+1}"):
            batch = pair_data[i:i+BATCH_SIZE_TR]
            anchors, positives = zip(*batch)

            seqs = list(anchors) + list(positives)

            enc = tokenizer(
                seqs, padding=True, truncation=True,
                max_length=MAX_LENGTH, return_tensors="pt"
            ).to(device)

            with torch.cuda.amp.autocast(enabled=USE_FP16):
                with torch.no_grad():
                    out = backbone(**enc, output_hidden_states=True)

                emb = model(out.hidden_states, enc["attention_mask"])
                B = len(batch)

                ea, ep = emb[:B], emb[B:]
                sim = (ea @ ep.T) / TEMPERATURE
                labels = torch.arange(B, device=device)

                loss = (F.cross_entropy(sim, labels) +
                        F.cross_entropy(sim.T, labels)) / 2

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

            epoch_losses.append(loss.item())

        print(f"📉 Epoch {epoch+1} Loss = {np.mean(epoch_losses):.4f}")

    # -------------------------------------------------
    # 6) 추론
    # -------------------------------------------------
    print("\n🔍 test 임베딩 생성")

    outputs = []

    for i in tqdm(range(0, len(test_sequences), BATCH_SIZE_INFER)):
        batch = test_sequences[i:i+BATCH_SIZE_INFER]

        view1 = batch
        view2 = [reverse_complement(x) for x in batch]

        seqs = view1 + view2

        enc = tokenizer(
            seqs, padding=True, truncation=True,
            max_length=MAX_LENGTH, return_tensors="pt"
        ).to(device)

        with torch.no_grad():
            out = backbone(**enc, output_hidden_states=True)
            emb_all = model(out.hidden_states, enc["attention_mask"])

            B = len(batch)
            emb_mean = (emb_all[:B] + emb_all[B:]) / 2.0
            outputs.append(emb_mean.cpu().numpy())

    embeddings = np.concatenate(outputs, axis=0)

    # -------------------------------------------------
    # 7) 저장
    # -------------------------------------------------
    print("\n💾 결과 저장")

    col_names = [f"emb_{i:04d}" for i in range(OUTPUT_DIM)]
    df_emb = pd.DataFrame(embeddings, columns=col_names)

    final = pd.concat([test_df[["ID"]], df_emb], axis=1)
    final.to_csv(OUT_PATH, index=False)

    print("🎉 완료! 파일 저장됨 →", OUT_PATH)


if __name__ == "__main__":
    main()


🔥 GPU: NVIDIA GeForce RTX 4060

📌 외부데이터 로드 (GRCh38 + CHM13)
👉 전체 외부 데이터 개수: 9502199
🔻 샘플링: 120000

📌 InfoNCE pair 생성
👉 test 시퀀스 개수: 13711


c:\Users\USER\anaconda3\envs\glm\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\USER\.cache\huggingface\hub\models--InstaDeepAI--nucleotide-transformer-v2-500m-multi-species. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
A new version of the following files was downloaded from https://huggingface.co/


🚀 InfoNCE 학습 시작


EPOCH 1:   0%|          | 0/938 [00:00<?, ?it/s]C:\Users\USER\AppData\Local\Temp\ipykernel_18796\2743362785.py:211: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=USE_FP16):
EPOCH 1: 100%|██████████| 938/938 [05:44<00:00,  2.72it/s]


📉 Epoch 1 Loss = 0.1248


...

📉 Epoch 2 Loss = 0.0080


EPOCH 3: 100%|██████████| 938/938 [06:00<00:00,  2.60it/s]


📉 Epoch 3 Loss = 0.0050

🔍 test 임베딩 생성


100%|██████████| 429/429 [13:27<00:00,  1.88s/it]



💾 결과 저장
🎉 완료! 파일 저장됨 → C:\Users\USER\Desktop\전지예\submission_infonce_v4.csv
